<a href="https://colab.research.google.com/github/shahriariit/COVID-19-Xray/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

import numpy as np

import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import resnet50, inception_v3, vgg16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.optimizers import Adam
import numpy as np
import keras
from keras.applications.vgg16 import preprocess_input
#from keras.applications.resnet50 import preprocess_input
#from keras.applications.inceptionv3 import preprocess_input
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from sklearn.metrics import accuracy_score 
from sklearn.metrics import f1_score 
from imblearn.metrics import geometric_mean_score 
from imblearn.metrics import sensitivity_score 
from imblearn.metrics import specificity_score 
from sklearn.preprocessing import label_binarize 
from sklearn.metrics import roc_auc_score 
from sklearn.model_selection import cross_val_predict 





Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / function

In [0]:
from PIL import Image
import cv2
import os
import csv

TRAIN_DATADIR= r'/content/drive/My Drive/Data/train'
CATEGORIES =  ["COVID_19","NORMAL","Viral_Pneumonia"]
#CATEGORIES =  ["COVID19","NORMAL","PNEUMONIA"]

IMG_SIZE = 100
TEST_DATADIR = r'/content/drive/My Drive/Data/test'
training_data = []
testing_data= []
new_array = []

def create_training_data():
    #i=-1
    for category in CATEGORIES:
        train_path = os.path.join(TRAIN_DATADIR, category)
        test_path = os.path.join(TEST_DATADIR,category)
        class_num = np.asarray(CATEGORIES.index(category))
        #i+=1
        for img in os.listdir(train_path):
            #print(img)
            img_path = train_path + '/'+ img
            # img_file.show()
            print('=================TRAIN DATA===============')
           
            img = image.load_img(img_path, target_size=(IMG_SIZE,IMG_SIZE))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            print('Input image shape:', x.shape)
        
            training_data.append(x)
        for img in os.listdir(test_path):
            #print(img)
            img_path = test_path + '/'+ img
            # img_file.show()
            print('=================Test DATA===============')
           
            img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            print('Input image shape:', x.shape)
        
            training_data.append(x)
                
create_training_data()             
 
                                    


=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input image shape: (1, 100, 100, 3)
=================TRAIN DATA===============
Input ima

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
img_data = np.array(training_data)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

(855, 1, 100, 100, 3)
(1, 855, 100, 100, 3)
(855, 100, 100, 3)


In [0]:
num_classes = 3
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')
labels[0:230]=0
labels[230:460]=1
labels[460:690]=2
labels[690:745]=0
labels[745:800]=1
labels[800:855]=2
names = ['COVID19','NORMAL','PNEUMONIA']
# convert class labels to on-hot encoding
y = keras.utils.to_categorical(labels, num_classes)



In [0]:
from sklearn.utils import shuffle
X,y = shuffle(img_data,y, random_state=2)
print(X.shape)
print(y.shape)

(855, 100, 100, 3)
(855, 3)


In [0]:
# Fine Tune with sequential MODEL 
def vgG():  
    import keras
    conv_base  = vgg16.VGG16(weights='imagenet',include_top=False,input_shape = (100,100,3))
    model = keras.models.Sequential()
    model.add(conv_base)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64 , activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.Dropout(0.5))

    #model.add(keras.layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
    #model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(3, activation='softmax'))

    # Make last block of the conv_base trainable:

    for layer in conv_base.layers[:25]:
       layer.trainable = False
    for layer in conv_base.layers[25:]:
       layer.trainable = True 

    print('Last block of the conv_base is now trainable') 
    #print(model.summary())
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy']) 
    return model 

In [0]:
# Fine Tune with sequential MODEL 
def resnet():  
    import keras
    conv_base  = resnet50.ResNet50(weights='imagenet',include_top=False,input_shape = (100,100,3))
    model = keras.models.Sequential()
    model.add(conv_base)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64 , activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.Dropout(0.5))

    #model.add(keras.layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
    #model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(3, activation='softmax'))

    # Make last block of the conv_base trainable:

    for layer in conv_base.layers[:165]:
       layer.trainable = False
    for layer in conv_base.layers[165:]:
       layer.trainable = True 

    print('Last block of the conv_base is now trainable')
    #print(model.summary())
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
# Fine Tune with sequential MODEL 
def inception():  
    import keras
    conv_base  = keras.applications.inception_v3.InceptionV3(weights='imagenet',include_top=False,input_shape = (100,100,3))
    model = keras.models.Sequential()
    model.add(conv_base)
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(64 , activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
    model.add(keras.layers.Dropout(0.5))

    #model.add(keras.layers.Dense(256, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
    #model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(3, activation='softmax'))

    # Make last block of the conv_base trainable:

    for layer in conv_base.layers[:249]:
       layer.trainable = False
    for layer in conv_base.layers[249:]:
       layer.trainable = True 

    print('Last block of the conv_base is now trainable')
    #print(model.summary())
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
model =vgG()
model.summary()

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold,  cross_val_score
from sklearn.model_selection import cross_val_predict 


rs50 = KerasClassifier(build_fn =vgG, epochs=50, batch_size=32, verbose=0)

#X,y = shuffle(X_train,y_train)
#DNN.fit(X_train,y_train)
y_pred = cross_val_predict(estimator = rs50, X = X, y = y, cv = 10)

#results = cross_val_score(estimator, X_train, y_train, cv=10)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


# Predicting the Test set results

print(y_pred.shape)
y =np.argmax(y,axis=1)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)
print("Confusion Matrix: ",cm )
print('Accuracy :       ', accuracy_score(y, y_pred)) 
print('ROC :            ', roc_auc_score(label_binarize(y, classes=[0,1,2]), 
                                        label_binarize(y_pred, classes=[0,1,2]), average = 'weighted'))
print('Precision :      ', precision_score(y, y_pred, average='weighted')) 
print('Recall :         ', recall_score(y, y_pred, average='weighted')) 
print('F-Measure :      ', f1_score(y, y_pred, average = 'weighted'))
print('Geometric Mean : ', geometric_mean_score(y, y_pred, average = 'weighted'))
print('Sensitivity :    ', sensitivity_score(y, y_pred, average = 'weighted'))
print('Specificity :    ', specificity_score(y, y_pred, average = 'weighted'))
print('Type I Error :   ', (1-geometric_mean_score(y, y_pred, average = 'weighted')))
print('Type II Error :  ', (1-specificity_score(y, y_pred, average = 'weighted'))) 


In [0]:
import plotly.express as px
import pandas as pd 

maximum= {'metrics':['Accuracy','AUC','Precision','Recall','F-Measure','Specificity','Type I error','Type II error'],
           'score':[0.9403,0.9552,0.9403,0.9403,0.9403,0.9701,0.0448,0.0298]}

average = {'metrics': ['Accuracy','AUC','Precision','Recall','F-Measure','specificity','Type I error','Type II error'],
           'score':[0.8169,0.8627,0.8202,0.8169,0.8151,0.9084,0.1394,0.0915]} 
maximum = pd.DataFrame(data = maximum,index=range(8))
average = pd.DataFrame(data = average,index=range(8))


In [0]:
fig = px.bar(maximum[['metrics', 'score']], 
             y="score", x="metrics", color='metrics',text='score',range_y = [0,1], 
              template='ggplot2', title='Average Score')
fig.show()
    